1. finnhub에서 가져온 뉴스 데이터 확인

In [2]:
import pandas as pd

df = pd.read_csv("./AAPL_news_2025-06-07.csv")

In [3]:
df.head(5)

,title,summary,link,publisher,pubDate,source
0,"GameStop earnings, US-China talks, May CPI: Wh...",Market Domination Overtime hosts Julie Hyman a...,https://finnhub.io/api/news?id=c58f1563b03c26f...,Yahoo,2025-06-06 23:00:00,finnhub
1,"Dow Jones Futures: S&P 500 Nears Highs, 5 Stoc...","Dow Jones Futures: S&P 500 Nears Highs, 5 Stoc...",https://finnhub.io/api/news?id=9bbe0c1e8527045...,DowJones,2025-06-06 22:58:53,finnhub
2,Why Apple is falling behind & how it can pull ...,"Patrick Moorhead, founder and CEO at Moor Insi...",https://finnhub.io/api/news?id=b7251bb62e2351a...,Yahoo,2025-06-06 22:00:00,finnhub
3,Apple Set For Low-Key WWDC News Cycle After AI...,Apple's WWDC 2025 is likely to be a low-key af...,https://finnhub.io/api/news?id=f1baa303fafbf4a...,Yahoo,2025-06-06 21:31:53,finnhub
4,Where Will Apple Stock Be in 5 Years?,Apple shares have been a monster winner in the...,https://finnhub.io/api/news?id=d95edfa32b01336...,Yahoo,2025-06-06 21:18:00,finnhub


In [4]:
df['title'][0]

'GameStop earnings, US-China talks, May CPI: What to Watch'

In [5]:
df['summary'][0]

"Market Domination Overtime hosts Julie Hyman and Josh Lipton preview the top market stories and headlines Wall Street will be paying attention to next week, including corporate earnings from companies like GameStop (GME), Oracle (ORCL), Adobe (ADBE), and the J.M. Smucker Company (SJM); Apple's (AAPL) 2025 Worldwide Developers Conference (WWDC) kicking off; May's inflation data; and the launch of Tesla's (TSLA) robotaxi beta in Austin, Texas, starting on June 12. To watch more expert insights and analysis on the latest market action, check out more Market Domination Overtime\xa0here."

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226 entries, 0 to 225
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      226 non-null    object
 1   summary    222 non-null    object
 2   link       226 non-null    object
 3   publisher  226 non-null    object
 4   pubDate    226 non-null    object
 5   source     226 non-null    object
dtypes: object(6)
memory usage: 10.7+ KB


In [9]:
df['pubDate'][0]


'2025-06-06 23:00:00'

In [11]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

MODEL = "yiyanghkust/finbert-tone"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
finbert = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, return_all_scores=True)

df = pd.read_csv("AAPL_news_2025-06-07.csv") 
df = df.fillna("")

df["text"] = df["title"] + ". " + df["summary"]
texts = df["text"].tolist()

results = finbert(texts, batch_size=8)

pos_scores, neu_scores, neg_scores = [], [], []

for res in results:
    d = {r["label"].lower(): r["score"] for r in res}
    pos_scores.append(d.get("positive", 0.0))
    neu_scores.append(d.get("neutral", 0.0))
    neg_scores.append(d.get("negative", 0.0))

df["finbert_positive"] = pos_scores
df["finbert_neutral"] = neu_scores
df["finbert_negative"] = neg_scores

output_csv = "apple_finbert_finnhub.csv"
df.to_csv(output_csv, index=False)
print(f"결과 저장 완료: {output_csv}")


Device set to use cuda:0
c:\Users\jhh33\anaconda3\envs\pythonProject1\Lib\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


✅ 결과 저장 완료: apple_finbert_finnhub.csv
